## Strategy
-  Model architectured to go wider and not deeper.
-  Defined tf records class.
-  Reached the target accuracy of 85%

In [1]:
import tensorflow as tf

In [2]:
print(tf.__version__)

1.15.0


In [0]:
from tensorflow.keras.datasets import cifar10
import sys
from tensorflow.keras import layers

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [0]:
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

In [0]:
class TFRecords:   
#   def __init__(self, out_filename, images, labels):
#     self.out_filename = out_filename
#     self.images = images
#     self.labels = labels

  def _int64_feature(self, value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

  def _bytes_feature(self, value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
    
  def createDataRecord(self, out_filename, images, labels):

    writer = tf.io.TFRecordWriter(out_filename)

    for i in range(len(images)):
      feature = {
          'image_raw': self._bytes_feature(images[i].tostring()),
          'label': self._int64_feature(labels[i])
      }

      example = tf.train.Example(features=tf.train.Features(feature=feature))

      writer.write(example.SerializeToString())

    writer.close()
    sys.stdout.flush()

In [0]:
# Instantite the class

tfRecords = TFRecords()

In [0]:
tfRecords.createDataRecord('train.tfrecords', x_train, y_train)

In [0]:
tfRecords.createDataRecord('test.tfrecords', x_test, y_test)

In [10]:
!ls

sample_data  test.tfrecords  train.tfrecords


In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 5
num_filter = 22
compression = 0.5
dropout_rate = 0.2

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 22, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(5):
        BatchNorm = layers.BatchNormalization()(temp)
        relu = layers.Activation('relu')(BatchNorm)
        Conv2D_3_3 = layers.Conv2D(int(num_filter), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = layers.Dropout(dropout_rate)(Conv2D_3_3)
        concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 22, dropout_rate = 0.2):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = layers.Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = layers.Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    AvgPooling = layers.AveragePooling2D(pool_size=(2,2))(relu)
    flat = layers.Flatten()(AvgPooling)
    output = layers.Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 22
dropout_rate = 0.2
l = 5
input = layers.Input(shape=(img_height, img_width, channel,))
First_Conv2D = layers.Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

In [30]:
model = tf.keras.Model(inputs=[input], outputs=[output])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_52 (Conv2D)              (None, 32, 32, 22)   594         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_52 (BatchNo (None, 32, 32, 22)   88          conv2d_52[0][0]                  
__________________________________________________________________________________________________
activation_52 (Activation)      (None, 32, 32, 22)   0           batch_normalization_52[0][0]     
____________________________________________________________________________________________

In [0]:
model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [32]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_52 (Conv2D)              (None, 32, 32, 22)   594         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_52 (BatchNo (None, 32, 32, 22)   88          conv2d_52[0][0]                  
__________________________________________________________________________________________________
activation_52 (Activation)      (None, 32, 32, 22)   0           batch_normalization_52[0][0]     
____________________________________________________________________________________________

In [0]:
def parser(record):
  keys_to_features = {
      'image_raw': tf.FixedLenFeature((), tf.string),
      'label': tf.FixedLenFeature((), tf.int64)
  }
  parsed = tf.parse_single_example(record, keys_to_features)
  image = tf.decode_raw(parsed['image_raw'], tf.uint8)
  image = tf.cast(image, tf.int32)
  image = tf.reshape(image, [32,32,3])
  label = tf.cast(parsed["label"], tf.int32)
  label = tf.one_hot(label, 10)
  return image, label

In [0]:
def get_dataset_from_TFRecord(filename):
  dataset = tf.data.TFRecordDataset(filenames='train.tfrecords')
  dataset = dataset.map(parser)
  return dataset    

In [0]:
dataset = get_dataset_from_TFRecord('train.tfrecords')
dataset = dataset.batch(512)
dataset = dataset.repeat()

In [0]:
val_dataset = get_dataset_from_TFRecord('test.tfrecords')

In [0]:
val_dataset = val_dataset.batch(512).repeat()

In [38]:
model.fit(dataset, epochs=50, steps_per_epoch=30,
          validation_data=val_dataset,validation_steps=3)

Train on 30 steps, validate on 3 steps
Epoch 1/50
30/30 [==============================] - 27s 911ms/step - loss: 1.9842 - acc: 0.2595 - val_loss: 3.2236 - val_acc: 0.1133
Epoch 2/50
30/30 [==============================] - 18s 589ms/step - loss: 1.6350 - acc: 0.3832 - val_loss: 2.0027 - val_acc: 0.2344
Epoch 3/50
30/30 [==============================] - 18s 588ms/step - loss: 1.5326 - acc: 0.4279 - val_loss: 1.8375 - val_acc: 0.3242
Epoch 4/50
30/30 [==============================] - 20s 664ms/step - loss: 1.4176 - acc: 0.4760 - val_loss: 1.5731 - val_acc: 0.4245
Epoch 5/50
30/30 [==============================] - 18s 588ms/step - loss: 1.3611 - acc: 0.5021 - val_loss: 1.3859 - val_acc: 0.4896
Epoch 6/50
30/30 [==============================] - 18s 588ms/step - loss: 1.3273 - acc: 0.5117 - val_loss: 1.5501 - val_acc: 0.4440
Epoch 7/50
30/30 [==============================] - 18s 584ms/step - loss: 1.2608 - acc: 0.5370 - val_loss: 1.2996 - val_acc: 0.5293
Epoch 8/50
30/30 [============

In [39]:
model.fit(dataset, epochs=50, steps_per_epoch=30,
          validation_data=val_dataset,validation_steps=3)

Train on 30 steps, validate on 3 steps
Epoch 1/50
30/30 [==============================] - 23s 781ms/step - loss: 0.5829 - acc: 0.7927 - val_loss: 0.7560 - val_acc: 0.7474
Epoch 2/50
30/30 [==============================] - 18s 587ms/step - loss: 0.6038 - acc: 0.7894 - val_loss: 0.6978 - val_acc: 0.7604
Epoch 3/50
30/30 [==============================] - 18s 589ms/step - loss: 0.6179 - acc: 0.7801 - val_loss: 0.7244 - val_acc: 0.7578
Epoch 4/50
30/30 [==============================] - 18s 585ms/step - loss: 0.5782 - acc: 0.7959 - val_loss: 0.6073 - val_acc: 0.8034
Epoch 5/50
30/30 [==============================] - 18s 588ms/step - loss: 0.5866 - acc: 0.7892 - val_loss: 0.8821 - val_acc: 0.7318
Epoch 6/50
30/30 [==============================] - 18s 588ms/step - loss: 0.5898 - acc: 0.7923 - val_loss: 0.9512 - val_acc: 0.7038
Epoch 7/50
30/30 [==============================] - 18s 585ms/step - loss: 0.5568 - acc: 0.8026 - val_loss: 0.5893 - val_acc: 0.7943
Epoch 8/50
30/30 [============